In [1]:
%matplotlib inline
import numpy as np
np.random.seed(2016)
import matplotlib.pyplot as plt

import os
import glob
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from scipy.misc import imread
from scipy.misc import imresize

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import __version__ as keras_version

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
def load_cropped_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('cropped_data', fld, '*.jpg')
        files = sorted(glob.glob(path))
        for fl in files:
            flbase = os.path.basename(fl)
            img = image.load_img(fl, target_size=(224, 224))
            img = image.img_to_array(img)
            X_train.append(img)
            X_train_id.append(fld + '/' + flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

def read_and_normalize_cropped_train_data():
    train_data, train_target, train_id = load_cropped_train()

    print('Convert to numpy...')
    train_data = np.array(train_data)
    train_target = np.array(train_target)

    print('Convert to float...')
    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 8)

    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id

In [3]:
train_data, train_target, train_id = read_and_normalize_cropped_train_data()

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder OTHER (Index: 4)
Load folder SHARK (Index: 5)
Load folder YFT (Index: 6)
Read train data time: 36.3 seconds
Convert to numpy...
Convert to float...
('Train shape:', (4371, 224, 224, 3))
(4371, 'train samples')


In [4]:
base_model = ResNet50(weights='imagenet', include_top = False, input_shape=(224,224,3))

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(8, activation='softmax')(x)
model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='sgd', loss='categorical_crossentropy')

In [5]:
batch_size = 16
nb_epoch = 30
random_state = 51

X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.2, random_state=0)

In [6]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),]
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True,
          verbose=2, validation_data=(X_test, y_test), callbacks=callbacks)

Train on 3496 samples, validate on 875 samples
Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[16,3,230,230]
	 [[Node: conv1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](zero_padding2d_1/Pad, conv1/kernel/read)]]
	 [[Node: mul_2/_1623 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2622_mul_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'conv1/convolution', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/a/h/jli04/Envs/deep-venv/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/a/h/jli04

In [ ]:
predictions_valid = model.predict(X_test.astype('float32'), batch_size=batch_size, verbose=2)
score = log_loss(y_test, predictions_valid)

In [ ]:
print "log loss score: ", score

In [ ]:
from sklearn.metrics import accuracy_score
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(predictions_valid, axis = 1)
acc = accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)

In [ ]:
print "accuracy: ", acc

In [ ]:
print y_true[0:35]
print y_pred[0:35]

print y_true[35:70]
print y_pred[35:70]